# Augmenting vggnet with new classes

## Load Data

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

/home/youi/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Loading images and labels
((train_ds, train_labels), (test_ds, test_labels)), dataset_info = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[70%:]"], ## Train test split
    shuffle_files=True,
    with_info=True,
    batch_size=-1,
    as_supervised=True,  # Include labels
)


In [4]:
print(train_ds.shape)
print(test_ds.shape)

(2569, 442, 1024, 3)
(1101, 441, 640, 3)


In [5]:
dataset_info.features['label'].names

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [6]:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.imshow(test_ds[0].numpy().astype("uint8"))
# #plt.title(test_labels[0])
# plt.axis("off")

In [7]:
## Resizing images
train_ds = tf.image.resize(train_ds, (224, 224))
test_ds = tf.image.resize(test_ds, (224, 224))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [8]:
## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

## Load model

In [9]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False)#, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

In [10]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=True, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

In [11]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### create headless model

In [16]:
headless_model=tf.keras.models.Sequential()

In [17]:
for layer in base_model.layers[:-1]:
    headless_model.add(layer)

In [18]:
headless_model.trainable=False

In [19]:
# train_ds.shape[1:]

### create finetune model

In [20]:
headless_inputs=tf.keras.Input(shape=train_ds.shape[1:])

In [21]:
x=headless_model(headless_inputs)

In [23]:
outputs= tf.keras.layers.Dense(5)(x)

In [24]:
finetune_model=tf.keras.Model(headless_inputs,outputs)

In [25]:
for layer in finetune_model.layers:
    print('{:90s} {}'.format(str(layer),'trainable' if layer.trainable else 'non-trainable'))

<keras.engine.input_layer.InputLayer object at 0x7fa709312ec0>                             trainable
<keras.engine.sequential.Sequential object at 0x7fa709343a00>                              non-trainable
<keras.layers.core.dense.Dense object at 0x7fa6fdf161a0>                                   trainable


In [26]:
finetune_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 4096)              134260544 
                                                                 
 dense (Dense)               (None, 5)                 20485     
                                                                 
Total params: 134,281,029
Trainable params: 20,485
Non-trainable params: 134,260,544
_________________________________________________________________


In [28]:
finetune_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])


In [29]:
finetune_model.fit(train_ds,train_labels,epochs=10)

Epoch 1/10
81/81 [==============================] - 12s 108ms/step - loss: 1.1913 - categorical_accuracy: 0.5761
Epoch 2/10
81/81 [==============================] - 9s 106ms/step - loss: 0.7278 - categorical_accuracy: 0.7182
Epoch 3/10
81/81 [==============================] - 8s 105ms/step - loss: 0.6760 - categorical_accuracy: 0.7450
Epoch 4/10
81/81 [==============================] - 9s 105ms/step - loss: 0.5768 - categorical_accuracy: 0.7820
Epoch 5/10
81/81 [==============================] - 10s 126ms/step - loss: 0.4883 - categorical_accuracy: 0.8139
Epoch 6/10
81/81 [==============================] - 9s 105ms/step - loss: 0.4374 - categorical_accuracy: 0.8392
Epoch 7/10
81/81 [==============================] - 9s 106ms/step - loss: 0.4176 - categorical_accuracy: 0.8478
Epoch 8/10
81/81 [==============================] - 8s 104ms/step - loss: 0.3736 - categorical_accuracy: 0.8645
Epoch 9/10
81/81 [==============================] - 8s 104ms/step - loss: 0.3416 - categorical_accurac

In [30]:
results = finetune_model.evaluate(test_ds,test_labels)

35/35 [==============================] - 5s 116ms/step - loss: 0.9705 - categorical_accuracy: 0.7184


In [31]:
import matplotlib.pyplot as plt

### Rebuild vgg model with new classes

can be done in two ways:
- using concatenate layer, requires the use of a functional model (the method we use)
- using weight transfer (via get_weights() and set_weights()) 

In [36]:
x=tf.keras.Input(shape=train_ds.shape[1:])
headless_inputs=x

In [37]:
for layer in base_model.layers[:-1]:
    x=layer(x)

In [38]:
# concat headless_outputs w/ base_model outputs
new_outputs=tf.keras.layers.Concatenate()([base_model.layers[-1](x),finetune_model.layers[-1](x)])
new_outputs=tf.nn.softmax(new_outputs)

In [39]:
new_outputs

<KerasTensor: shape=(None, 1005) dtype=float32 (created by layer 'tf.nn.softmax')>

In [41]:
#new_model.add(new_outputs)#
new_model=tf.keras.Model(headless_inputs,new_outputs)

In [42]:
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           multiple             0           ['input_4[0][0]']                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_2[1][0]']                
                                )                                                                 
                                                                                            

In [43]:
new_model.layers[-1].output_shape

(None, 1005)

In [44]:
test_image=tf.expand_dims(train_ds[0], axis=0)

In [46]:
res=new_model(test_image)

In [58]:
tf.math.argmax(res,axis=1)[0].numpy()

1001

In [56]:
tf.math.argmax(train_labels[0]).numpy()

1

### additional tasks
now that the model is complete, better accuracy can be obtained by unfreezing the classification network and fine-tuning it on a dataset containing all labels.
other utility functions can also be implemented (e.g. decode_predictions)